In [5]:
!pip install pypdf transformers langchain huggingface_hub faiss-cpu

     -------------------------------------- 249.3/249.3 kB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 10.8/10.8 MB 9.4 MB/s eta 0:00:00
     ------------------------------------- 323.6/323.6 kB 19.6 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB ? eta 0:00:00
     ---------------------------------------- 92.7/92.7 kB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 61.7/61.7 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 49.1/49.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.8.3
    Uninstalling numexpr-2.8.3:
      Successfully uninstalled numexpr-2.8.3


## Import statements

In [36]:
import os
from dotenv import load_dotenv
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

## Setting up hugging face account

In [5]:
load_dotenv()

hf_hub = os.environ.get("hf_hub")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_hub


In [19]:
path_to_file = 'Quotes.pdf'
pdf_loader = PyPDFLoader(path_to_file)
pages = pdf_loader.load_and_split()

In [20]:
pages[0]

Document(page_content='Anime Quotes About Life  \n"The world isn\'t perfect. But it\'s there for us, doing the best it can that\'s what makes it so damn beautiful." \n--- Roy Mustang (Full Metal Alchemist)  \n"To know sorrow is not terrifying. What is terrifying is to know you can\'t go back to happiness you could h\nave." --- Matsumoto Rangiku (Bleach)  \n"We are all like fireworks: We climb, we shine and always go our separate ways and become further apart\n. But even when that time comes, let\'s not disappear like a firework and continue to shine forever." --- Hits\nugaya Toshiro (Bleach)  \n"Those who stand at the top determine what\'s wrong and what\'s right! This very place is neutral ground! J\nustice will prevail, you say? But, of course, it will! Whoever wins this war becomes justice!" --- Don Quixot\ne Doflamingo (One Piece)  \n"Fear is not evil. It tells you what weakness is. And once you know your weakness, you can become stron\nger as well as kinder." --- Gildarts Clive (F

## Embedding the input paragraph

In [22]:
embeddings = HuggingFaceHubEmbeddings()
db = FAISS.from_documents(pages, embeddings)

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


## Similarity Search

In [23]:
query = "Can you quote a quote told by Yami"
docs = db.similarity_search(query)
docs

[Document(page_content='Anime Quotes About Life  \n"The world isn\'t perfect. But it\'s there for us, doing the best it can that\'s what makes it so damn beautiful." \n--- Roy Mustang (Full Metal Alchemist)  \n"To know sorrow is not terrifying. What is terrifying is to know you can\'t go back to happiness you could h\nave." --- Matsumoto Rangiku (Bleach)  \n"We are all like fireworks: We climb, we shine and always go our separate ways and become further apart\n. But even when that time comes, let\'s not disappear like a firework and continue to shine forever." --- Hits\nugaya Toshiro (Bleach)  \n"Those who stand at the top determine what\'s wrong and what\'s right! This very place is neutral ground! J\nustice will prevail, you say? But, of course, it will! Whoever wins this war becomes justice!" --- Don Quixot\ne Doflamingo (One Piece)  \n"Fear is not evil. It tells you what weakness is. And once you know your weakness, you can become stron\nger as well as kinder." --- Gildarts Clive (

## Conversational Chain

In [24]:
model = 'google/flan-t5-large'
model_kwargs = {'temperature': 0.4}

In [28]:
chain = load_qa_chain(HuggingFaceHub(repo_id = model, model_kwargs=model_kwargs), chain_type='stuff')
qa = ConversationalRetrievalChain.from_llm(HuggingFaceHub(repo_id=model, model_kwargs=model_kwargs), db.as_retriever())

In [30]:
chat_history =[]
topic = 'Anime Quotes'
print(f'Hi Hello I am a custom chat bot designed to answer on {topic}')
while True:
    question = input('User: ')
    if question.lower() == 'exit':
        print(f'Thank You for using {topic} bot :)')
        break
    
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    
    print("Chatbot: ",result['answer'])

Hi Hello I am a custom chat bot designed to answer on Anime Quotes
User: Hi
Chatbot:  Anime Quotes About Life
User: Quote by Gildarts
Chatbot:  "The world isn't perfect. But it's there for us, doing the
User: Yami
Chatbot:  Son Goku
User: Son Goku
Chatbot:  None
User: Quote by Goku
Chatbot:  Don Quixote Doflamingo
User: exit
Thank You for using Anime Quotes bot :)


## The models output is not so good so we need to make some improvements in the data as well as tune hyperparameters of the model

In [31]:
chat_history

[('Hi', 'Anime Quotes About Life'),
 ('Quote by Gildarts',
  '"The world isn\'t perfect. But it\'s there for us, doing the'),
 ('Yami', 'Son Goku'),
 ('Son Goku', 'None'),
 ('Quote by Goku', 'Don Quixote Doflamingo')]

## Prompt Method

In [37]:
chat_history =[]
topic = 'Anime Quotes'
print(f'Hi Hello I am a custom chat bot designed to answer on {topic}')
while True:
    question = input('User: ')
    if question.lower() == 'exit':
        print(f'Thank You for using {topic} bot :)')
        break
    
    hub_llm = HuggingFaceHub(repo_id=model, model_kwargs=model_kwargs)
    template = """
    Question: {question}
    Answer:
    """
    prompt = PromptTemplate(template=template, input_variables=['question'])
    llm_chain = LLMChain(prompt=prompt, llm=hub_llm)
    result = llm_chain.run(question)
    chat_history.append((question, result))
    
    print("Chatbot: ",result)

Hi Hello I am a custom chat bot designed to answer on Anime Quotes
User: Hi
Chatbot:  I am a student at the University of California, Berkeley.
User: Who won the IPL 2023 trophy
Chatbot:  ahli jio
User: What is the color of orange fruit
Chatbot:  yellow
User: Which IPL franchise has more torphies
Chatbot:  ahli jio
User: Which IPL team has won most number of tropies
Chatbot:  ahli jio
User: IPL full form
Chatbot:  icp
User: Best anime to watch
Chatbot:  ninja turtles
User: Are you connected to the internet
Chatbot:  no
User: can you tell me the question for previous answer
Chatbot:  What is the name of the first person to be killed in a car crash?
User: exit
Thank You for using Anime Quotes bot :)


## The model is not so good one and it is not connected to internet so it will hallucinate the answers